In [1]:
import pandas as pd
import numpy as np 
from chembl_webresource_client.new_client import new_client
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [2]:
def target_search(target_name):
    target = new_client.target
    res = target.filter(pref_name__icontains = target_name)
    if len(res) > 200:
        raise ValueError('Search Target too broad, Try Another Search Term.')
    else:
        targets = pd.DataFrame(res)
        return(targets) #Returns a df of targets matching your search term
    

In [3]:
df_target_search = target_search("Acetylcholinesterase")

In [4]:
def get_bioactivities_data(target_chembl_ID):
    activity = new_client.activity
    res = activity.filter(target_chembl_id=target_chembl_ID).filter(standard_type="IC50").only(['molecule_chembl_id','canonical_smiles','standard_value', 'standard_type'])
    df = pd.DataFrame(res[0:500])#Set a limit for now, take too long to compute over 500 activities 
    df['standard_value'] = df['standard_value'].astype(float)
    
    conditions = [
        (df['standard_value'] >= 10000),
        (df['standard_value'] <= 1000)
    ]
    choices = ['inactive', 'active']
    df['bioactivity_class'] = np.select(conditions, choices, default = 'intermediate')
    return(df)

In [5]:
df_bioactivities = get_bioactivities_data('CHEMBL220')

## Calculate Lipinski descriptors
Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the druglikeness of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the Rule-of-Five or Lipinski's Rule.

The Lipinski's Rule stated the following:

1. Molecular weight < 500 Dalton
2. Octanol-water partition coefficient (LogP) < 5
3. Hydrogen bond donors < 5
4. Hydrogen bond acceptors < 10

In [6]:
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [7]:
df_lipinski = lipinski(df_bioactivities.canonical_smiles)

In [8]:
df_combined = pd.concat([df_bioactivities, df_lipinski], axis = 1)
df_combined

,canonical_smiles,molecule_chembl_id,standard_type,standard_value,type,value,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors
0,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,CHEMBL133897,IC50,750.00,IC50,0.75,active,312.325,2.8032,0.0,6.0
1,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,CHEMBL336398,IC50,100.00,IC50,0.1,active,376.913,4.5546,0.0,5.0
2,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,CHEMBL131588,IC50,50000.00,IC50,50.0,inactive,426.851,5.3574,0.0,5.0
3,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,CHEMBL130628,IC50,300.00,IC50,0.3,active,404.845,4.7069,0.0,5.0
4,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,CHEMBL130478,IC50,800.00,IC50,0.8,active,346.334,3.0953,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
495,CCCCCCCNc1c2c(nc3cc(Cl)ccc13)CCCC2,CHEMBL59782,IC50,13.18,Log IC50,-7.88,active,330.903,6.1493,1.0,2.0
496,C[C@]12CC[C@H](O)[C@@](C)(CO)C1C[C@H](O)[C@@]1...,CHEMBL25629,IC50,35000.00,IC50,35.0,inactive,455.507,2.1922,3.0,8.0
497,CN1C(=O)Cc2cc3c(CCC4CCN(Cc5ccccc5)CC4)noc3cc21...,CHEMBL94546,IC50,0.48,IC50,0.48,active,485.606,3.6955,1.0,6.0
498,COc1ccc2c(c1)-c1nc3ccccc3c(N)c1C2,CHEMBL293842,IC50,4265.80,Log IC50,-5.37,intermediate,262.312,3.3968,1.0,3.0


## Convert IC50 to pIC50
To allow IC50 data to be more uniformly distributed, we will convert IC50 to the negative logarithmic scale which is essentially -log10(IC50).

This custom function pIC50() will accept a DataFrame as input and will:

Take the IC50 values from the standard_value column and converts it from nM to M by multiplying the value by 10
Take the molar value and apply -log10
Delete the standard_value column and create a new pIC50 column

In [9]:
# https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

In [10]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [11]:
df_norm = norm_value(df_combined)
df_norm

c:\Users\jacks\anaconda3\envs\drug_discovery\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


,canonical_smiles,molecule_chembl_id,standard_type,type,value,bioactivity_class,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
0,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,CHEMBL133897,IC50,IC50,0.75,active,312.325,2.8032,0.0,6.0,750.00
1,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,CHEMBL336398,IC50,IC50,0.1,active,376.913,4.5546,0.0,5.0,100.00
2,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,CHEMBL131588,IC50,IC50,50.0,inactive,426.851,5.3574,0.0,5.0,50000.00
3,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,CHEMBL130628,IC50,IC50,0.3,active,404.845,4.7069,0.0,5.0,300.00
4,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,CHEMBL130478,IC50,IC50,0.8,active,346.334,3.0953,0.0,6.0,800.00
...,...,...,...,...,...,...,...,...,...,...,...
495,CCCCCCCNc1c2c(nc3cc(Cl)ccc13)CCCC2,CHEMBL59782,IC50,Log IC50,-7.88,active,330.903,6.1493,1.0,2.0,13.18
496,C[C@]12CC[C@H](O)[C@@](C)(CO)C1C[C@H](O)[C@@]1...,CHEMBL25629,IC50,IC50,35.0,inactive,455.507,2.1922,3.0,8.0,35000.00
497,CN1C(=O)Cc2cc3c(CCC4CCN(Cc5ccccc5)CC4)noc3cc21...,CHEMBL94546,IC50,IC50,0.48,active,485.606,3.6955,1.0,6.0,0.48
498,COc1ccc2c(c1)-c1nc3ccccc3c(N)c1C2,CHEMBL293842,IC50,Log IC50,-5.37,intermediate,262.312,3.3968,1.0,3.0,4265.80


In [ ]:
df_final = pIC50(df_norm)

In [13]:
df_final.to_csv('data/cleaned_bioactivity_data.csv', index=False)